<a href="https://colab.research.google.com/github/elliemci/chatbots/blob/main/qa_chatbot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Wikipedia "Canned Response"

A type of FAQ bot that only know the anser to basic, general questions with a state, an added context menagment.

In [ ]:
# install Python transformers development version with
!pip install transformers[sentencepiece] pyTelegramBotAPI wikipedia

In [2]:
import telebot
import wikipedia
from transformers import pipeline

print(wikipedia.summary("Wikipedia"))

The Wikimedia Foundation, Inc. (WMF) is an American 501(c)(3) nonprofit organization headquartered in San Francisco, California, and registered as a charitable foundation under local laws. It is best known as the host platform for Wikipedia, the largest crowdsourced online encyclopedia in the world. The organization also hosts other related projects and MediaWiki, a wiki software.The Wikimedia Foundation was established in 2003 in St. Petersburg, Florida, by Jimmy Wales as a nonprofit way to fund Wikipedia, Wiktionary, and other crowdsourced wiki projects that had until then been hosted by Bomis, Wales's for-profit company. The Foundation finances itself mainly through millions of small donations from Wikipedia readers, collected through email campaigns and annual fundraising banners placed on Wikipedia and its sister projects. These are complemented by grants from philanthropic organizations and tech companies, and starting in 2022, by services income from Wikimedia Enterprise.
The Fo

In [24]:
import os
from getpass import getpass

os.environ['TELEGRAM_BOT_TOKEN'] = getpass('Enter your bot token: ')
TOKEN = os.getenv('TELEGRAM_BOT_TOKEN')

# create the bot an instance of TeleBot when set the token
bot = telebot.TeleBot(TOKEN)

Enter your bot token: ··········


In [25]:
# handles incoming /start and /help
@bot.message_handler(commands=['start', 'help'])
def send_welcome(message):
	bot.reply_to(message, "Howdy, how are you doing?")

# execute command with /greet
@bot.message_handler(commands=['greet'])
def greet(message):
  bot.reply_to(message, "What a wondeful day! Hows it going?")

# reply without repeating the question or phrase
@bot.message_handler(commands=['hello'])
def hello(message):
  bot.send_message(message.chat.id, "Hello there!")

# echos all incoming text messages
#@bot.message_handler(func=lambda m: True)
#def echo_all(message):
  #bot.reply_to(message, message.text)

In [ ]:
state = 0
subject = ""
qa_pipeline = pipeline("question-answering")

@bot.message_handler(func=lambda message: True)
def message(message):
    global state, subject
    chat_id = message.chat.id

    if state == 0:
      bot.send_message(chat_id, "Hi, tell me a subject you want ot talk about?")
      state = 1
    elif state == 1:
        subject = message.text
        if subject == "no":
          bot.send_message(chat_id, "Goodbye!")
          state = 0
        else:
          bot.send_message(chat_id, "Ask me a question about this topic.")
          state = 2
    elif state == 2:
        question = message.text
        try:
            context = wikipedia.summary(subject)
        except wikipedia.exceptions.DisambiguationError as e:
            bot.send_message(chat_id, "Your query is ambiguous, please provide more specific subject.")
            return
        except wikipedia.exceptions.PageError as e:
            bot.send_message(chat_id, "I couldn't find a Wikipedia page on that subject.")
            return
        answer = qa_pipeline({"question": question, "context": context})["answer"]
        bot.send_message(chat_id, answer)
        state = 3
    elif state == 3:
      bot.send_message(chat_id, "Anything else you want to talk about?")
      state = 1

bot.infinity_polling()